In [20]:
%pip install -r ./requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [21]:
%pip install -r ./yolov5/requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [ ]:
!python yolov5 / train.py --img 640 --batch 16 --epochs 2 --data yolov5 / data / coco128.yaml --weights pretrained_models / yolov5n.pt --cache ram --device cpu --project./ trained_models --name xd

In [22]:
import pandas as pd

In [23]:
df = pd.read_csv('datasets/test_db/annotations.csv')

In [24]:
df

,filename,width,height,label,xmin,ymin,xmax,ymax
0,0.jpg,416,416,black-bishop,280,227,310,284
1,0.jpg,416,416,black-king,311,110,345,195
2,0.jpg,416,416,black-queen,237,85,262,159
3,0.jpg,416,416,black-rook,331,277,366,333
4,0.jpg,416,416,black-rook,235,3,255,51
...,...,...,...,...,...,...,...,...
104,6.jpg,416,416,black-rook,335,297,368,355
105,6.jpg,416,416,black-queen,234,97,260,173
106,6.jpg,416,416,black-pawn,273,84,291,128
107,6.jpg,416,416,black-pawn,204,118,223,162


In [49]:
grouped_df = df.groupby(['filename']).agg({
    'xmin': list,
    'ymin': list,
    'xmax': list,
    'ymax': list,
    'label': list
})

In [50]:
grouped_df

,xmin,ymin,xmax,ymax,label
filename,,,,,
0.jpg,"[280, 311, 237, 331, 235, 267, 271, 280, 283, ...","[227, 110, 85, 277, 3, 38, 72, 145, 188, 284, ...","[310, 345, 262, 366, 255, 286, 291, 303, 305, ...","[284, 195, 159, 333, 51, 82, 116, 190, 233, 33...","[black-bishop, black-king, black-queen, black-..."
1.jpg,"[298, 207, 284, 306, 339, 291, 325, 275, 240, ...","[38, 109, 137, 85, 297, 10, 199, 121, 115, 18,...","[318, 227, 305, 336, 369, 313, 354, 293, 264, ...","[93, 163, 206, 163, 355, 59, 255, 164, 172, 59...","[black-bishop, black-bishop, black-queen, blac..."
2.jpg,"[243, 333, 234, 271, 202, 129, 101, 167, 91]","[172, 296, 93, 83, 118, 128, 15, 118, 254]","[277, 367, 259, 291, 223, 161, 124, 189, 120]","[256, 355, 174, 127, 165, 213, 60, 164, 303]","[black-king, black-rook, black-queen, black-pa..."
3.jpg,[208],[88],[226],[128],[black-pawn]
4.jpg,"[302, 292, 320, 251, 236, 203, 204, 297, 274, ...","[81, 10, 194, 241, 110, 139, 87, 301, 119, 202...","[336, 316, 350, 279, 260, 230, 223, 321, 295, ...","[167, 61, 249, 306, 169, 214, 133, 351, 166, 2...","[black-king, black-rook, black-rook, black-bis..."
5.jpg,[174],[153],[196],[210],[black-bishop]
6.jpg,"[170, 131, 167, 136, 101, 94, 257, 335, 234, 2...","[79, 129, 119, 52, 15, 256, 268, 297, 97, 84, ...","[195, 162, 189, 158, 123, 118, 291, 368, 260, ...","[132, 213, 165, 97, 59, 304, 354, 355, 173, 12...","[white-rook, white-king, white-pawn, white-paw..."


SKRYPT KONWERTUJĄCY CSV NA YOLO

Poniższe ścieżki należy uzupełnić. Dodatkowo plik `classes.names` należy utworzyć zgodnie ze schematem:
```text
class_name_0
class_name_1
class_name_2
class_name_3
class_name_4
class_name_5
```

In [51]:
# Sciezka do starych anotacji (labeli) w formacie csv
csv_path = 'datasets/test_db/annotations.csv'
# Sciezka do obrazow
images_path = 'datasets/test_db/images'
# Sciezka do pliku z nazwami klas (yolo wymaga ID klasy, a nie nazwy!)
classes_names_path = 'datasets/test_db/classes.names'
# Sciezka do zapisu datasetu w nowym formacie
dataset_path = 'datasets/test_db_converted'

In [53]:
from collections import defaultdict
from pathlib import Path
from shutil import copy

from PIL import Image

classes_names_file = Path(classes_names_path)
classes_names_list = classes_names_file.read_text(encoding='utf8').split()
class_name_id_map = {class_name: idx for idx, class_name in enumerate(classes_names_list)}

images_old_dir = Path(images_path)
dataset_dir = Path(dataset_path)
images_new_dir = dataset_dir / 'images'
images_new_dir.mkdir(exist_ok=True, parents=True)
labels_dir = dataset_dir / 'labels'
labels_dir.mkdir(exist_ok=True, parents=True)

csv_file = Path(csv_path)
csv_content = csv_file.read_text(encoding='utf8')
csv_lines = csv_content.split()
csv_rows = [line.split(',') for line in csv_lines]
header_row = csv_rows[0]
# Mapowanie nazw kolumn na index (kolejność) na liście z nagłówkiem
col_idx = {col_name: idx for idx, col_name in enumerate(header_row)}
data_rows = csv_rows[1:]

file_bounds_map = defaultdict(list)

for row in data_rows:
    filename = str(row[col_idx['filename']])
    class_name = row[col_idx['label']]
    xmin = int(row[col_idx['xmin']])
    ymin = int(row[col_idx['ymin']])
    xmax = int(row[col_idx['xmax']])
    ymax = int(row[col_idx['ymax']])

    # Format yolo:
    # class x_center y_center width height

    image_file = images_old_dir / filename
    img_w, img_h = Image.open(image_file).size

    w = xmax - xmin
    h = ymax - ymin
    x_cnt = (xmin + xmax) / 2.0
    y_cnt = (ymin + ymax) / 2.0

    norm_bounds = [x_cnt / img_w, y_cnt / img_h, w / img_w, h / img_h]
    bounds_str = ' '.join(f'{i:.6f}' for i in norm_bounds)

    yolo_fmt_str = f'{class_name_id_map.get(class_name, 0)} {bounds_str}'
    file_bounds_map[filename].append(yolo_fmt_str)

for filename, bounds_list in file_bounds_map.items():
    image_old_file = images_old_dir / filename
    image_new_file = images_new_dir / filename
    copy(image_old_file, image_new_file)

    label_file_name = image_new_file.with_suffix('.txt').name
    label_file = labels_dir.joinpath(label_file_name)
    label_file_content = '\n'.join(bounds_list)
    label_file.write_text(label_file_content, encoding='utf8')

Splitowanie folderów zawierających obrazki oraz labelki na foldery `train` / `val` / `test`.

In [48]:
images_dir = 'datasets/test_db_converted/images'
labels_dir = 'datasets/test_db_converted/labels'

seed = 1337
ratio = (.8, 0.1, 0.1)

In [47]:
from shutil import move
import random

random.seed(seed)  # make sure its reproducible

for directory_path in [images_dir, labels_dir]:
    directory = Path(directory_path)
    files = [f for f in directory.iterdir() if f.is_file() and not f.name.startswith(".")]
    files.sort()
    random.shuffle(files)

    # the data was shuffled already
    split_train_idx = int(round(ratio[0] * len(files)))
    split_val_idx = split_train_idx + int(round(ratio[1] * len(files)))

    use_test = len(ratio) == 3

    files_train = files[:split_train_idx]
    files_val = files[split_train_idx:split_val_idx] if use_test else files[split_train_idx:]

    li = [(files_train, "train"), (files_val, "val")]

    # optional test folder
    if use_test:
        files_test = files[split_val_idx:]
        li.append((files_test, "test"))

    for files, folder_type in li:
        end_path = directory / folder_type
        Path(end_path).mkdir(parents=True, exist_ok=True)
        for f in files:
            move(str(f), str(end_path))